<a href="https://colab.research.google.com/github/SergioCarmo-ro/monit_aluno_tempo_real/blob/main/monit_aluno_tempo_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PROJETO MONITORAMENTO DE ALUNOS EM TEMPO REAL**

In [1]:
!pip install numpy
!pip install matplotlib
!pip install tensorflow
!pip install pyrealsense2
!pip install mtcnn
!pip install opencv-python

In [2]:
import tensorflow as tf
import cv2
import numpy as np
from scipy.io import loadmat, savemat
from matplotlib import pyplot as plt
import pandas as pd
import cv2
from PIL import Image
import mtcnn
from mtcnn.mtcnn import MTCNN
from matplotlib.patches import Rectangle
from os import listdir
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import seaborn as sns
# Importe funções do repo (ex.: utils, renderer)

In [3]:
import tensorflow as tf

# Create an empty graph definition
graph_def = tf.compat.v1.GraphDef()

# Specify the path to your protobuf file
protobuf_file_path = 'path/to/your/model.pb'

# Open the protobuf file and parse its content into the graph definition
try:
    with tf.io.gfile.GFile(protobuf_file_path, 'rb') as f:
        graph_def.ParseFromString(f.read())

    # Import the graph definition into the default TensorFlow graph
    tf.import_graph_def(graph_def, name='')

    print(f"Successfully loaded graph definition from '{protobuf_file_path}'")

except tf.errors.NotFoundError:
    print(f"Error: The file '{protobuf_file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Error: The file 'path/to/your/model.pb' was not found.


In [4]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Specify the folder path in Google Drive
folder_path = '/content/drive/My Drive/'

# List files in the folder
try:
    for fn in os.listdir(folder_path):
        file_path = os.path.join(folder_path, fn)
        if os.path.isfile(file_path):
            file_size = os.path.getsize(file_path)
            print('Found file "{name}" with length {length} bytes'.format(
                name=fn, length=file_size))
        elif os.path.isdir(file_path):
            print('Found directory "{name}"'.format(name=fn))

except FileNotFoundError:
    print(f"Error: The folder '{folder_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found directory "Colab Notebooks"
Found directory "pthread (1)"
Found directory "pthread"
Found directory "Classroom"
Found file "CM_SérgioAdemirRochadoCarmo.pdf" with length 27319 bytes
Found file "He_SérgioAdemirRochado Carmo.pdf" with length 11314 bytes
Found file "CAPA TRAB. EVENTUAL 1.pdf" with length 157891 bytes
Found file "Atividade Eventual 1 - FTC.pdf" with length 176190 bytes
Found directory "Atividade p N2"
Found directory "Ativ. P  Prova N3"
Found file "Trabalho 2.pdf" with length 513550 bytes
Found directory "Documentos meus "
Found file "report (9).pdf" with length 26929 bytes
Found file "Certificado.pdf" with length 201687 bytes
Found file "Certificado Curva ABC.pdf" with length 201170 bytes
Found file "Certificado Introdução a Linguagem Kotlen .pdf" with length 200410 bytes
Found file "algoritmo Bio Inspirados.pdf" with length 200756 byte

In [5]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Specify the folder path in Google Drive
folder_path = '/content/drive/My Drive/Estagio/'

# List files in the folder
try:
    for fn in os.listdir(folder_path):
        file_path = os.path.join(folder_path, fn)
        if os.path.isfile(file_path):
            file_size = os.path.getsize(file_path)
            print('Found file "{name}" with length {length} bytes'.format(
                name=fn, length=file_size))
except FileNotFoundError:
    print(f"Error: The folder '{folder_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Função para reconstruir e extrair embedding
def reconstruct_face(image_path):
    # Pré-processar imagem (resize para 224x224, normalize)
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = (img / 255.0 - 0.5) * 2.0  # Normalização para [-1, 1]
    img = np.expand_dims(img, axis=0)

    with tf.Session() as sess:
        coeff = sess.run('coeff:0', feed_dict={'input:0': img})  # Coeficientes (shape, exp, tex, etc.)
        # Extrair embedding: Use coeff como feature vector para comparação
        embedding = coeff[:, :257]  # Exemplo: identity + expression
        return embedding

In [7]:
# Exemplo de comparação (para reconhecimento)
def compare_faces(emb1, emb2):
    similarity = np.dot(emb1, emb2.T) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similarity > 0.7  # Threshold ajustável

In [8]:
# Exemplo de comparação (para reconhecimento)
def compare_faces(emb1, emb2):
    similarity = np.dot(emb1, emb2.T) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return similarity > 0.7  # Threshold ajustável

In [9]:
#AQUI CÓDIGO PARA CAMERA 3D PLUGADA NO EQUIPAMENTO
# Real-time com câmera 3D (ex.: RealSense para depth)
#import pyrealsense2 as rs
#import numpy as np
#import cv2

#pipeline = rs.pipeline()
#config = rs.config()
#config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
#config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

#try:
 #   pipeline.start(config)
  #  while True:
   #     frames = pipeline.wait_for_frames()
    #    color_frame = frames.get_color_frame()
     #   depth_frame = frames.get_depth_frame()
      #  if not color_frame or not depth_frame:
       #     continue
        #color_img = np.asanyarray(color_frame.get_data())
        # Salve temp e reconstrua (integre depth para 3D: combine RGB com depth map)
        #cv2.imwrite('temp.jpg', color_img)
        #emb = reconstruct_face('temp.jpg')
        # Compare com database de embeddings de alunos
        #print("Aluno reconhecido?")
        # Treinamento: Use script do repo para fine-tune em seu dataset (adapte train.py se disponível)
#except RuntimeError as e:
 #   print(f"Error starting camera pipeline: {e}")
  #  print("Please ensure your RealSense camera is connected.")




In [ ]:
# Usando a camera do notebook
# Captura uma imagem da câmera do notebook (via navegador)
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
from io import BytesIO

def take_photo(filename='foto.jpg', quality=0.8):
    js_code = '''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const capture = document.createElement('button');
          capture.textContent = ' Capturar';
          div.appendChild(capture);
          document.body.appendChild(div);
          const video = document.createElement('video');
          video.style.display = 'block';
          const stream = await navigator.mediaDevices.getUserMedia({video: true});
          document.body.appendChild(video);
          video.srcObject = stream;
          await video.play();
          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
          await new Promise((resolve) => capture.onclick = resolve);
          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          const context = canvas.getContext('2d');
          context.drawImage(video, 0, 0, canvas.width, canvas.height);
          stream.getTracks().forEach(track => track.stop());
          div.remove();
          video.remove();
          return canvas.toDataURL('image/jpeg', quality);
        }

        takePhoto(''' + str(quality) + ''');
    '''
    data = eval_js(js_code) # Evaluate the Javascript code string
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# Captura e mostra a imagem
filename = take_photo()
print("Imagem capturada:", filename)
Image.open(filename)

* TESTES
- Adicionais de funcionamentos  
- Outros ajustes

In [ ]:
# Só uma breve verificação da versão
# Célula 1 - execute no Colab
# Instalações
!pip install mtcnn==0.1.1 tensorflow==2.14.1 opencv-python-headless==4.7.0.72 keras-preprocessing --quiet

# Monte o Google Drive (se o dataset está no Drive)
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import sys
print(sys.version)

In [ ]:
!pip install tensorflow==2.18.0

In [ ]:
!pip install tensorflow==2.18.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
# Create a sample DataFrame (replace this with your actual data)
df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})
# Exemplo: salvar um DataFrame
df.to_csv('/content/drive/MyDrive/meu_arquivo.csv')

In [ ]:
# Célula 2 - ajustar o caminho do dataset abaixo
import os
from pathlib import Path

DATASET_ROOT = Path('/content/drive/My Drive/Estagio/')  # ajuste aqui

# checar estrutura
if not DATASET_ROOT.exists():
    raise FileNotFoundError(f"Dataset não encontrado em {DATASET_ROOT}. Ajuste o caminho.")

classes = [p.name for p in DATASET_ROOT.iterdir() if p.is_dir()]
print(f"Encontradas {len(classes)} classes (identidades). Exemplos: {classes[:10]}")


In [ ]:
from mtcnn import MTCNN
from PIL import Image
import cv2
from tqdm import tqdm
import numpy as np
from pathlib import Path

detector = MTCNN()

# Aqui está o ajuste principal:
PROCESSED_ROOT = Path('/content/processed_rosto_alunos')
PROCESSED_ROOT.mkdir(exist_ok=True)  # cria a pasta se não existir

for cls in classes:
    src_dir = DATASET_ROOT / cls
    out_dir = PROCESSED_ROOT / cls  # Save processed images to a subdirectory named after the class
    out_dir.mkdir(parents=True, exist_ok=True)
    for img_file in tqdm(list(src_dir.glob('*')), desc=f"Processando {cls}"):
        try:
            img = cv2.imread(str(img_file))
            if img is None:
                continue
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = detector.detect_faces(img_rgb)
            if results:
                best = max(results, key=lambda x: x['box'][2]*x['box'][3])
                x, y, w, h = best['box']
                x, y = max(0, x), max(0, y)
                crop = img_rgb[y:y+h, x:x+w]
                face = cv2.resize(crop, (160,160))
                Image.fromarray(face).save(out_dir / img_file.name)
        except Exception as e:
            pass

print(" Extração de faces concluída! Faces salvas em:", PROCESSED_ROOT)

In [ ]:
!pip install --upgrade --no-deps --force-reinstall mtcnn

In [ ]:
!pip install lz4

In [ ]:
import os

PROCESSED_ROOT = "/content/processed_rosto_alunos"

# Verifica se o diretório existe
print("Existe:", os.path.exists(PROCESSED_ROOT))

# Lista os arquivos encontrados
for root, dirs, files in os.walk(PROCESSED_ROOT):
    print(f"\n Pasta: {root}")
    for f in files:
        print("  -", f)


In [ ]:
import os
import shutil
import tensorflow as tf
from pathlib import Path
import time

# Caminhos
src_dir = "/content/processed_rosto_alunos/rostos_CSV"
dst_dir = "/content/processed_rosto_alunos/treinamento/rostos_alunos"
target_dir = os.path.join(dst_dir, "classe_unica")

# Check if the source directory is empty and provide a helpful message
if not os.listdir(src_dir):
    print(f"Error: Source directory is empty: {src_dir}")
    print("Please run the face extraction step (cell 3cFPtBLV_3WW) first and ensure it successfully saves images to this directory.")
else:
    # List contents of src_dir to help identify the correct image folder
    print(f"Contents of {src_dir}: {os.listdir(src_dir)}")

    # Cria a structure esperada: uma subpasta dentro de outra
    os.makedirs(target_dir, exist_ok=True)

    # Mover ou copiar as imagens
    copied_count = 0
    for img in os.listdir(src_dir):
        if img.lower().endswith((".png", ".jpg", ".jpeg")):
            shutil.copy(os.path.join(src_dir, img), os.path.join(target_dir, img))
            copied_count += 1

    print(f"Imagens reorganizadas com sucesso! Total copiado: {copied_count}")

    # Add a small delay
    time.sleep(1)

    # Check if images are in the target directory AFTER the delay
    target_dir_contents = os.listdir(target_dir)
    print(f"Conteúdo de {target_dir}: {target_dir_contents}")

    # Create the dataset manually from a list of file paths
    image_files = [os.path.join(target_dir, f) for f in os.listdir(target_dir) if f.lower().endswith((".png", ".jpg", ".jpeg"))]

    if not image_files:
        print(f"Error: No image files found in the target directory: {target_dir}")
        print("Please check if the images were copied correctly.")
    else:
        list_ds = tf.data.Dataset.from_tensor_slices(image_files)
        list_ds = list_ds.shuffle(len(image_files), seed=42)

        def process_path(file_path):
            img = tf.io.read_file(file_path)
            # Try decoding as both JPEG and PNG
            try:
                img = tf.image.decode_jpeg(img, channels=3)
            except tf.errors.InvalidArgumentError:
                img = tf.image.decode_png(img, channels=3)

            img = tf.image.convert_image_dtype(img, tf.float32)
            img = tf.image.resize(img, [224, 224])
            # Since we have only one class, the label is always 0
            label = 0
            return img, label

        train_ds = list_ds.map(process_path)

        # Set up the dataset for training
        BATCH_SIZE = 32
        train_ds = train_ds.batch(BATCH_SIZE)
        train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

        print(" Dataset created successfully using tf.data.Dataset.from_tensor_slices!")

In [ ]:
import glob

# Procura todas as imagens dentro das subpastas
imagens = glob.glob("/content/processed_rosto_alunos/**/*.*", recursive=True)
imagens_png = [img for img in imagens if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

print(f"Total de imagens encontradas: {len(imagens_png)}")
print("\nAlguns exemplos:")
for img in imagens_png[:10]:
    print(img)


In [ ]:
#  criando o treinamento

import tensorflow as tf
from pathlib import Path
import os

# Point to the directory containing the images for the single class
IMAGE_DIR = Path('/content/processed_rosto_alunos/treinamento/rostos_alunos/classe_unica')
IMAGE_SIZE = (160,160)
BATCH_SIZE = 32
SEED = 42

# Create a list of image file paths manually
image_files = [str(f) for f in IMAGE_DIR.glob('*') if f.suffix.lower() in ['.png', '.jpg', '.jpeg']]

if not image_files:
    print(f"Error: No image files found in the directory: {IMAGE_DIR}")
else:
    list_ds = tf.data.Dataset.from_tensor_slices(image_files)
    list_ds = list_ds.shuffle(len(image_files), seed=SEED)

    def process_path(file_path):
        img = tf.io.read_file(file_path)
        # Try decoding as both JPEG and PNG
        try:
            img = tf.image.decode_jpeg(img, channels=3)
        except tf.errors.InvalidArgumentError:
            img = tf.image.decode_png(img, channels=3)

        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, IMAGE_SIZE)
        # Since we have only one class, the label is always 0
        label = 0
        return img, label

    train_ds = list_ds.map(process_path)

    # Set up the dataset for training
    train_ds = train_ds.batch(BATCH_SIZE)
    train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    # Manually set class name and number of classes
    class_names = ["classe_unica"]
    num_classes = len(class_names)

    print(f" Dataset pronto com {tf.data.experimental.cardinality(train_ds).numpy() * BATCH_SIZE} imagens na classe:", class_names)

    # val_ds is not created with this approach, set to None
    val_ds = None

In [ ]:
from tensorflow.keras import layers, models

# Ensure that the dataset creation cell (TZALb_SZEzJe) has been run
# as it defines the dataset structure and IMAGE_SIZE.

# Explicitly set num_classes based on the dataset created in the previous cell (TZALb_SZEzJe)
# Assuming a single class ("classe_unica") is being used.
num_classes = 1

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.08),
])

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMAGE_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # congela camadas base

model = models.Sequential([
    data_augmentation,
    # Removed tf.keras.applications.mobilenet_v2.preprocess_input
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Ensure that the dataset creation cell (TZALb_SZEzJe) has been run
# as it defines the 'train_ds' and 'val_ds' variables.

# Explicitly check if train_ds is defined
try:
    train_ds is not None
except NameError:
    print("Error: train_ds is not defined.")
    print("Please run the dataset creation cell (TZALb_SZEzJe) first.")
    raise # Re-raise the error to stop execution

CKPT_PATH = '/content/face_classifier_mobilenetv2.h5'

callbacks = [
    ModelCheckpoint(CKPT_PATH, monitor='val_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

EPOCHS = 15

# Only include validation_data if val_ds is not None
if 'val_ds' in locals() and val_ds is not None:
    history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks)
else:
    # If val_ds is not defined or is None, train without validation data
    print("Note: val_ds is not available, training without validation data.")
    history = model.fit(train_ds, epochs=EPOCHS, callbacks=callbacks)

In [ ]:
base_model.trainable = True
fine_tune_at = 100  # libera apenas as últimas camadas

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

EPOCHS_FINE = 10
history_fine = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_FINE, callbacks=callbacks)


In [ ]:
!find /content -type f \( -iname "*.jpg" -o -iname "*.png" \) | head -n 20


In [ ]:
from mtcnn import MTCNN
import cv2
import matplotlib.pyplot as plt

img_path = str(list(DATASET_ROOT.rglob("*.jpg"))[0])  # pega a primeira imagem original
print("Exemplo de imagem:", img_path)

img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
detector = MTCNN()
faces = detector.detect_faces(img_rgb)

print(f"Detectadas {len(faces)} faces")
if faces:
    x, y, w, h = faces[0]['box']
    cv2.rectangle(img_rgb, (x,y), (x+w,y+h), (0,255,0), 2)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()
